#### 安裝套件

In [14]:
!pip install datasets evaluate jiwer librosa
!pip install --upgrade bitsandbytes transformers==4.50.0 accelerate
!pip install ctranslate2==4.4.0 whisperx
!apt-get install libcudnn8 libcudnn8-dev

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libcudnn8 is already the newest version (8.9.7.29-1+cuda12.2).
libcudnn8-dev is already the newest version (8.9.7.29-1+cuda12.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [16]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### 載入函式庫與參數設定

In [6]:
import os
import pandas as pd
import torch
import numpy as np
from datasets import Dataset, Audio
from transformers import (
    WhisperProcessor,
    WhisperForConditionalGeneration,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
)
from dataclasses import dataclass
from typing import Any, Dict, List, Union
import evaluate
from tqdm import tqdm
from transformers import EarlyStoppingCallback
from ctranslate2.converters import TransformersConverter
import whisperx
from transformers import WhisperProcessor, WhisperTokenizer
import json
import zipfile
from jiwer import wer

In [7]:
def setup_directories(version: str = "v99"):
    # Create result directory and version subdirectory
    result_dir = "model_result"
    result_dir = os.path.join(result_dir, "task1")
    version_dir = os.path.join(result_dir, version)
    model_dir = os.path.join(version_dir, "model")

    # Create directories if they don't exist
    os.makedirs(result_dir, exist_ok=True)
    os.makedirs(version_dir, exist_ok=True)
    os.makedirs(model_dir, exist_ok=True)
    print(result_dir, version_dir, model_dir)
    return version_dir, model_dir

In [28]:
# Setup directories
version = "v1"
version_dir, model_dir = setup_directories(version)


# Check CUDA availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

# 設定模型路徑
final_model_path = os.path.join(model_dir, "whisper-large-v3")
final_ct2_model_path = os.path.join(model_dir, "whisper-large-v3-ct2")
print(f"Model saved to {final_model_path}")

# load fine-tuning whisper
zip_model_path = "/content/drive/MyDrive/whisper-large-v3.zip"
extract_model_path = "/content/model_result/task1/v1/model/whisper-large-v3"
with zipfile.ZipFile(zip_model_path, 'r') as zip_ref:
  zip_ref.extractall(extract_model_path)

model_result/task1 model_result/task1/v1 model_result/task1/v1/model
Using device: cuda
GPU: Tesla T4
Model saved to model_result/task1/v1/model/whisper-large-v3


#### 資料前處理

In [29]:
from datasets import Dataset, Audio
import os

def prepare_dataset_from_list(sorted_audio_paths):
    val_dict = {
        "audio": sorted_audio_paths,
        "text": ["" for _ in sorted_audio_paths],  # 空字串
    }
    val_dataset = Dataset.from_dict(val_dict)
    val_dataset = val_dataset.cast_column("audio", Audio(sampling_rate=16000))
    return {"val": val_dataset}


In [31]:
def prepare_features(batch, processor):
    # Process audio
    audio = batch["audio"]

    # Compute log-mel input features from input audio array
    batch["input_features"] = processor.feature_extractor(
        audio["array"],
        sampling_rate=audio["sampling_rate"],
        language="en",   # or "zh"
        return_tensors="pt"
    ).input_features[0]

    # Encode target text to label ids 並限制最大長度為 448
    batch["labels"] = processor.tokenizer(
        batch["text"],
        max_length=448,
        truncation=True,
        padding="max_length",

    ).input_ids
    batch["file_path"] = batch["audio"]["path"]
    batch["audio_file_name"] = os.path.splitext(os.path.basename(batch["audio"]["path"]))[0]

    return batch

In [11]:
pip install natsort

In [32]:
import os
import zipfile
from natsort import natsorted

zip_path = "/content/drive/MyDrive/Private_dataset.zip"
extract_dir = "/content/audio_dataset"
os.makedirs(extract_dir, exist_ok=True)

# 解壓縮 ZIP（加入密碼）
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.setpassword(b"aicup20660205")  # ⚠️ 密碼要是 bytes
    zip_ref.extractall(path=extract_dir)

# 遞迴搜尋並排序 .wav 檔
audio_files = []
for root, dirs, files in os.walk(extract_dir):
    for file in files:
        if file.lower().endswith('.wav'):
            audio_files.append(os.path.join(root, file))

audio_files = natsorted(audio_files)
print(f"Found {len(audio_files)} audio files.")
for f in audio_files:
    print(f)

# 這裡假設你有定義 prepare_dataset_from_list 和 prepare_features
dataset = prepare_dataset_from_list(audio_files)

from transformers import WhisperProcessor
processor = WhisperProcessor.from_pretrained("/content/model_result/task1/v1/model/whisper-large-v3")

processed_dataset = {}
for split in ["val"]:
    print(f"Processing {split} dataset...")
    processed_dataset[split] = dataset[split].map(
        lambda x: prepare_features(x, processor=processor),
        remove_columns=dataset[split].column_names,
        num_proc=1
    )


Found 710 audio files.
/content/audio_dataset/private/60014.wav
/content/audio_dataset/private/60015.wav
/content/audio_dataset/private/60018.wav
/content/audio_dataset/private/60022.wav
/content/audio_dataset/private/60049.wav
/content/audio_dataset/private/60079.wav
/content/audio_dataset/private/60084.wav
/content/audio_dataset/private/60102.wav
/content/audio_dataset/private/60147.wav
/content/audio_dataset/private/60167.wav
/content/audio_dataset/private/60180.wav
/content/audio_dataset/private/60184.wav
/content/audio_dataset/private/60197.wav
/content/audio_dataset/private/60226.wav
/content/audio_dataset/private/60239.wav
/content/audio_dataset/private/60243.wav
/content/audio_dataset/private/60245.wav
/content/audio_dataset/private/60253.wav
/content/audio_dataset/private/60267.wav
/content/audio_dataset/private/60275.wav
/content/audio_dataset/private/60286.wav
/content/audio_dataset/private/60290.wav
/content/audio_dataset/private/60330.wav
/content/audio_dataset/private/603

Map:   0%|          | 0/710 [00:00<?, ? examples/s]

In [35]:
import glob

audio_dir = "/content/audio_dataset/private"
audio_paths = sorted(glob.glob(os.path.join(audio_dir, "*.wav")))  # 或是 .mp3, .flac, 視情況而定

dataset = prepare_dataset_from_list(audio_paths)


In [36]:
# Prepare dataset
processor = WhisperProcessor.from_pretrained("/content/model_result/task1/v1/model/whisper-large-v3")
# "audio_dataset/sample/Training_Dataset",
# Process datasets
processed_dataset = {}
for split in ["val"]: # "train", "test",
    print(f"Processing {split} dataset...")
    processed_dataset[split] = dataset[split].map(
        lambda x: prepare_features(x, processor=processor),
        remove_columns= dataset[split].column_names,
        num_proc=1
    )

Processing val dataset...


Map:   0%|          | 0/710 [00:00<?, ? examples/s]

#### 進行轉檔 Huggingface to Ctranslate2

In [37]:
def convert_hf_model_to_ct2(model_name_or_path: str, output_dir: str, quantization: str = "float32", trust_remote_code: bool = True, device: str = "cuda"):
    """
    Hugging Face convert to  CTranslate2

    parameter:
        model_name_or_path (str): Hugging Face moedel name or local directory.
        output_dir (str): Output directory where the CTranslate2 model is saved.
        quantization (str): Weight quantization scheme (possible values are: int8, int8_float32, int8_float16, int8_bfloat16, int16, float16, bfloat16, float32).
        trust_remote_code (bool): Allow converting models using custom code.
    """
    # 初始化轉換器
    converter = TransformersConverter(model_name_or_path,copy_files=["preprocessor_config.json", "tokenizer.json"] , trust_remote_code=trust_remote_code)
    # 執行轉換
    converter.convert(output_dir=output_dir,force=True)
    model = whisperx.load_model(output_dir, device=device)
    return model

In [38]:
print("Converting model to CTranslate2 format...")
model = convert_hf_model_to_ct2(
    model_name_or_path=final_model_path,
    output_dir=final_ct2_model_path,
    quantization="float16",
    trust_remote_code=True
)

Converting model to CTranslate2 format...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _speechbrain_save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _speechbrain_load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for load
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint save hook for _save
DEBUG:speechbrain.utils.checkpoints:Registered checkpoint load hook for _recover


No language specified, language will be first be detected for each audio file (increases inference time).
>>Performing voice activity detection using Pyannote...


INFO:pytorch_lightning.utilities.migration.utils:Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.5.1.post0. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint ../usr/local/lib/python3.11/dist-packages/whisperx/assets/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 3.3.2. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.6.0+cu124. Bad things might happen unless you revert torch to 1.x.


#### WhisperX進行轉錄

In [40]:
# Save predictions to TSV file  VALIDATION
def save_predictions_to_tsv(predictions,filenames, version_dir,export_json_file):
    task1_output_file = os.path.join(version_dir, "task1_answer.txt")
    output_file = os.path.join(version_dir, "val_results.txt")
    print("Saving predictions to file...")
    json_output_file = os.path.join(version_dir, "val_time_step.json")
    with open(json_output_file, "w", encoding="utf-8") as f:
        json.dump(export_json_file, f, ensure_ascii=False, indent=2)
    print(f"Results saved to {task1_output_file}")

In [45]:
def calculate_output(model, eval_dataloader, version_dir):
    print("Calculating ouput score...")
    predictions = []
    filenames = []
    export_json_file = {}

    i = 0
    for batch in tqdm(eval_dataloader, desc="處理評估資料", unit="batch"):
        with torch.no_grad():

            # Move batch to device
            audio = batch["file_path"]
                    # Generate predictions
            # 3. 先做轉錄，保留原本的 segments
            result = model.transcribe(audio, batch_size=1)

            if not result["segments"]:
              print(f"⚠️  No segments found for file: {audio}")
              continue  # 跳過這筆資料

            segments = result["segments"]  # 原本的句子層級 time-stamps
            transcript_dict = {
                "language": result["language"],
                "segments": []
            }
            # 4. 載入對齊模型（只做一次就好）
            align_model, metadata = whisperx.load_align_model(
                language_code=result["language"],
                device="cuda"
            )

            # 5. 做 forced-alignment，拿到 word_segments
            alignment = whisperx.align(segments, align_model, metadata, audio, device="cuda")

            # 6. 合併回原本 result
            result["word_segments"] = alignment["word_segments"]
            word_segments = alignment["word_segments"]

            for seg in segments:
                # 找出屬於這個 segment 的所有 word_segments
                words_in_seg = [
                    {
                        "word": w["word"],
                        "start": w["start"],
                        "end": w["end"],
                        "probability": w.get("probability", None)
                    }
                    for w in word_segments
                    if w["start"] >= seg["start"] and w["end"] <= seg["end"]
                ]
                transcript_dict["segments"].append({
                    "text": seg["text"],
                    "start": seg["start"],
                    "end": seg["end"],
                    "words": words_in_seg
                })

            # Decode predictions and references
            pred_str = result["segments"][0]["text"]
            export_json_file[batch["audio_file_name"]] = transcript_dict


            predictions.extend([pred_str])
            filenames.extend([batch["audio_file_name"]])  # 確保這行正確收集檔案名稱
            i += 1


    # Save predictions to file
    save_predictions_to_tsv(predictions,filenames, version_dir,export_json_file)

    return 0

In [43]:
print(f"Found {len(audio_paths)} audio files.")
if len(audio_paths) == 0:
    raise ValueError("No audio files found in the specified directory.")


Found 710 audio files.


In [46]:
calculate_output(model, processed_dataset["val"], version_dir)

Calculating ouput score...


處理評估資料:   0%|          | 0/710 [00:00<?, ?batch/s]

Detected language: en (1.00) in first 30s of audio...


處理評估資料:   0%|          | 1/710 [00:03<44:37,  3.78s/batch]

Detected language: en (0.98) in first 30s of audio...


...

/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
處理評估資料: 100%|█████████▉| 709/710 [43:21<00:04,  4.33s/batch]

Detected language: zh (0.99) in first 30s of audio...


/usr/local/lib/python3.11/dist-packages/transformers/configuration_utils.py:315: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(
處理評估資料: 100%|██████████| 710/710 [43:26<00:00,  3.67s/batch]


Saving predictions to file...
Results saved to model_result/task1/v1/task1_answer.txt


0